### 階層的なクラスタリング
https://wagtail.cds.tohoku.ac.jp/coda/python/machine-learning/clustering-2.html

- 最短距離法
  - クラスターA と　クラスターBに属する全てのデータ点ペアの距離の最小値
- 群平均法
  - 全てのペアについて等しい重みで取った平均
- Ward法
  - クラスターの「大きさ」、あるいは「広がり具合」の指標を考えてみる

1. 初期状態として、データ点はそれぞれデータ数1のクラスターと考える
2. クラスターのペアを調べ、距離が最小のペアを併合して一つのクラスターにまとめる
3. 2を、全体がひとつのクラスターになるまで繰り返す

#### デンドログラム（dendrogram)出力

In [ ]:
# coding: utf-8
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
# 生成するグループの数
k = 4
# グループ毎の分散
sigma = [[20, 0], [0, 20]]

# データ点の生成
x = np.empty((0, 2))
for ell in range(k):
  pc = np.random.uniform(low = -20, high = 20, size = (2,))  # グループ毎の代表点
  xs = np.random.multivariate_normal(pc, sigma, 30)
  x = np.concatenate([x, xs])

In [ ]:
# クラスタリング
link = linkage(x, 'ward')
labels = range(0, k*30)

# 結果のプロット
dendrogram(link,
           orientation = 'top',
           labels = labels,
           distance_sort = 'descending',
           color_threshold = 40.0,
           show_leaf_counts = True)
plt.show()

#### プロット
- コード中程で設定されている変数thresholdの値を超えたら、 そこでクラスターの併合を打ち切る

In [ ]:
# coding: utf-8
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage

In [ ]:
def rewrite_id(id, link, group, step, n):
  i = int(link[step, 0])
  j = int(link[step, 1])
  if i < n:
    group[i] = id
  else:
    rewrite_id(id, link, group, i - n, n)

  if j < n:
    group[j] = id
  else:
    rewrite_id(id, link, group, j - n, n)

In [ ]:
# 生成するグループの数
k = 4
# グループ毎の分散
sigma = [[20, 0], [0, 20]]

# データ点の生成
x = np.empty((0, 2))
for ell in range(k):
  pc = np.random.uniform(low = -20, high = 20, size = (2,))   # グループ毎の代表点
  xs = np.random.multivariate_normal(pc, sigma, 30)
  x = np.concatenate([x, xs])

In [ ]:
# クラスタリング

linked = linkage(x, 'ward')

n = x.shape[0]
threshold = 40
group = np.empty(n, dtype = 'int32')
step = 0
while True:
  if step >= n - 2:
    break
  dist = linked[step, 2]   
  if dist > threshold:
    break
  rewrite_id(step + n, linked, group, step, n)
  step = step +1 

In [ ]:
# 結果のプロット

cmap = plt.get_cmap("tab10")
cids = list(set(group))

print('cluster ids:', cids)

for i in range(x.shape[0]):
  ell = cids.index(group[i]) % 10
  plt.scatter(x[i, 0], x[i, 1], color = cmap(ell))

plt.grid(True)

plt.show()

### pythonで階層的クラスタリング
https://qiita.com/pontyo4/items/a2e7dec57c3699c519a5

#### 距離計算手法
method | 説明
:-----------|:------------
average | 重みのない平均距離
centroid | 重みのない重心までの距離
complete | 最大距離
median | 重みのある重心までの距離
single | 最小距離
ward | 内部平方距離
weighted | 重みのある平均距離

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt
import random

In [ ]:
# methodのリスト
method_list = ("average", "centroid", "complete", "median", "single", "ward", "weighted")
data = []   # dataを格納するリスト
label = []  # labelを格納するリスト

# dataを20個生成
for i in range(20):
  num = random.randint(0, 99)
  data.append(num)
  label.append(str(num))

In [ ]:
# DataFrameオブジェクト生成
df = pd.DataFrame(data)

# クラスタリング
for method in method_list:
  Z = linkage(df, method=method, metric="euclidean")
  dendrogram(Z, labels=label)
  plt.title(method)
  plt.show()

### 超シンプル_Python_階層クラスタリング_
https://qiita.com/D_U_8240/items/182a9e5bdfd4c7e2c9a7

- kaggle Titanicのデータを使用

In [ ]:
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd
import csv
import math
import string
import gensim
from gensim import corpora, models, similarities
import os
#下記が階層クラスタリングのライブラリ
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree

In [ ]:
df000 = pd.read_csv("https://raw.githubusercontent.com/jiai-edu/teaching_ep_elementary-phython/master/datasets/titanic_train.csv",encoding="CP932")
#欠損値状況を確認(アウトプットは省略しますので実行してみてください)
df000.isnull().sum()
#まずは年齢の欠損値を平均値で補完
df001 = df000
df001["Age"] = df000["Age"].fillna(df000["Age"].describe()["mean"])
#すいません何故がこのタイミングで説明変数を精査(必要なものをdrop)
df003 = df001.drop(["PassengerId","Name","Ticket","Cabin","Embarked"],axis=1)
#カテゴリ変数のダミー化(性別をダミー化します)
df004 = pd.get_dummies(df003["Sex"])
#ダミー化した性別情報を元のデータフレームに結合し、カテゴリ"Sex"を削除します
df005 = pd.concat([df003,df004],axis=1)
df006 = df005.drop("Sex",axis=1)

In [ ]:
#階層クラスタリングの実行
df007 = linkage(df006,metric = 'euclidean',method="ward")

In [ ]:
#階層結果のビジュアライズ
plt = dendrogram(df007)

In [ ]:
#cut_treeファンクションでクラスタを任意のクラスタに分けれるようにします

cuttree = cut_tree(df007)

#下記の例では3つのクラスタに分けます
df008 = cuttree[:, -3]  # 3クラスに分けるときに"-3"と書くところに注意

#元のファイルにクラスタ数を結合
df006["cluster"] = df008

#### 最適なクラスタを探すってどうやるの？
1. linkage()の際のmetricとmethodのパターンを変えてみる
2. 最後cuttree()のクラスタ数を変えてみる
3. そもそもインプットデータの特徴量を変えてみる

#### Ref.linkage()のチューニング要素
metric | 説明
:-----------|:------------
euclidean | ユークリッド距離
minkowski | ミンコフスキー距離
cityblock | マンハッタン距離
seuclidean | 標準化されたユークリッド距離
sqeuclidean | 乗ユークリッド距離
cosine | コサイン距離 (1からベクトルの余弦を引いたもの)
correlation | 層間距離(1から標本相関を引いたもの)
hamming | 異なる座標の比率を示すハミング距離
jaccard | 1からジャカード係数を引いた値
chebychev | チェビシェフ距離(最大座標差)
canbella | キャンベラ距離
braycurtis | ブレイ・カーティス距離
mahalanobis | マハラノビス距離



組合せは任意ですが、データが密集していてあまりうまく分けれない場合は\
metric = euclidean \
method = ward\
の組合せだとうまく分けれるケースが多い

### 階層的クラスタリングとシルエット係数
https://qiita.com/maskot1977/items/a35ac2fdc2c7448ee526

#### 実習用データ
https://qiita.com/maskot1977/items/453eddf5aa154c04668c

In [ ]:
'''
# URL によるリソースへのアクセスを提供するライブラリをインポートする。
import urllib.request 

# ウェブ上のリソースを指定する
url = 'https://raw.githubusercontent.com/maskot1977/ipython_notebook/master/toydata/sake_dataJ.txt'

# 指定したURLからリソースをダウンロードし、名前をつける。
urllib.request.urlretrieve(url, 'sake_dataJ.txt') 
'''

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/maskot1977/ipython_notebook/master/toydata/sake_dataJ.txt', sep='\t', index_col=0) # データの読み込み

In [ ]:
# 行列の正規化
dfs = df.apply(lambda x: (x-x.mean())/x.std(), axis=1).fillna(0)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# metric は色々あるので、ケースバイケースでどれかひとつ好きなものを選ぶ。
# method も色々あるので、ケースバイケースでどれかひとつ好きなものを選ぶ。
from scipy.cluster.hierarchy import linkage, dendrogram
result1 = linkage(dfs.iloc[:, :], 
                  #metric = 'braycurtis', 
                  #metric = 'canberra', 
                  #metric = 'chebyshev', 
                  #metric = 'cityblock', 
                  metric = 'correlation', 
                  #metric = 'cosine', 
                  #metric = 'euclidean', 
                  #metric = 'hamming', 
                  #metric = 'jaccard', 
                  #method= 'single')
                  method = 'average')
                  #method= 'complete')
                  #method='weighted')
plt.figure(figsize=(8, 8))
dendrogram(result1, orientation='right', labels=list(df.index), color_threshold=0.01)
plt.title("Dendrogram")
plt.xlabel("Threshold")
plt.grid()
plt.show()

In [ ]:
# 階層的クラスタリングの結果ファイル
pd.DataFrame(result1)

- 今回のデータは、沖縄県を除く４６都道府県からなので、０番目から４５番目までのノードから成ります
  - これらのノードは、メンバー数 1 のクラスタと見なすことができます

- 上の表の０行目は、２７番目のノードと２８番目のノード（ノード間距離＝0.000026）を結合して４６番目のノードにしたことを意味します
  - この新しく生成されたクラスタのメンバ数は 2 となります

- 同様に１行目は、３６番目のノードと３８番目のノード（ノード間距離＝0.000028）を結合して４７番目のノードにしたことを意味します
  - この新しいクラスタのメンバ数は 2 となります

- 同様にどんどん続けると、６行目で、２５番目のノードと４６番目のノードを結合します
  - この４６番目のノードは、０行目の結合で誕生したノード（クラスタ）です
  - この新しいクラスタのメンバ数は 3 となります

- ... というような、何と何をどういう順番で結合したかという情報が result1 に記述されています

#### Threshold を変えるとクラスタ数や平均クラスタサイズがどう変わるか調べる

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def draw_threshold_dependency(result):
  n_clusters = len(result)
  n_samples = len(result)
  df1 = pd.DataFrame(result)
  x1 = []
  y1 = []
  x2 = []
  y2 = []
  for i in range(len(result) - 1):
    n1 = int(result[i][0])
    n2 = int(result[i][1])
    val = result[i][2]
    n_clusters -= 1
    x1.append(val)
    x2.append(val)
    y1.append(n_clusters)
    y2.append(float(n_samples) / float(n_clusters))

  plt.subplot(2, 1, 1)
  plt.plot(x1, y1, 'yo-')
  plt.title('Threshold dependency of hierarchical clustering')
  plt.ylabel('Num of clusters')
  plt.subplot(2, 1, 2)
  plt.plot(x2, y2, 'ro-')
  plt.xlabel('Threshold')
  plt.ylabel('Ave cluster size')
  plt.show()

In [ ]:
draw_threshold_dependency(result1)

In [ ]:
# 指定した thoreshold でクラスタを得る関数を作る
def get_cluster_by_threshold(result, threshold):
  output_clusters = []
  output_cluster_ids = []
  x_result, y_result = result.shape
  n_clusters = x_result + 1
  cluster_id = x_result + 1
  father_of = {}
  x1 = []
  y1 = []
  x2 = []
  y2 = []
  for i in range(len(result) - 1):
    n1 = int(result[i][0])
    n2 = int(result[i][1])
    val = result[i][2]
    n_clusters -= 1
    if val < threshold:
      father_of[n1] = cluster_id
      father_of[n2] = cluster_id

    cluster_id += 1

  cluster_dict = {}
  for n in range(x_result + 1):
    if n not in father_of:
      output_clusters.append([n])
      continue

    n2 = n
    m = False
    while n2 in father_of:
      m = father_of[n2]
      #print [n2, m]
      n2 = m

    if m not in cluster_dict:
      cluster_dict.update({m:[]})
      cluster_dict[m].append(n)

  output_clusters += cluster_dict.values()

  output_cluster_id = 0
  output_cluster_ids = [0] * (x_result + 1)
  for cluster in sorted(output_clusters):
    for i in cluster:
      output_cluster_ids[i] = output_cluster_id
    output_cluster_id += 1

  return output_cluster_ids

In [ ]:
# 指定した thoreshold でクラスタを得る関数を使う。
clusterIDs = get_cluster_by_threshold(result1, 0.005)
print(clusterIDs)

In [ ]:
# 指定したクラスタ数でクラスタを得る関数を作る。
def get_cluster_by_number(result, number):
  output_clusters = []
  x_result, y_result = result.shape
  n_clusters = x_result + 1
  cluster_id = x_result + 1
  father_of = {}
  x1 = []
  y1 = []
  x2 = []
  y2 = []
  for i in range(len(result) - 1):
    n1 = int(result[i][0])
    n2 = int(result[i][1])
    val = result[i][2]
    n_clusters -= 1
    if n_clusters >= number:
      father_of[n1] = cluster_id
      father_of[n2] = cluster_id

    cluster_id += 1

  cluster_dict = {}
  for n in range(x_result + 1):
    if n not in father_of:
      output_clusters.append([n])
      continue

    n2 = n
    m = False
    while n2 in father_of:
      m = father_of[n2]
      #print [n2, m]
      n2 = m

    if m not in cluster_dict:
      cluster_dict.update({m:[]})
      cluster_dict[m].append(n)

  output_clusters += cluster_dict.values()

  output_cluster_id = 0
  output_cluster_ids = [0] * (x_result + 1)
  for cluster in sorted(output_clusters):
    for i in cluster:
      output_cluster_ids[i] = output_cluster_id
    output_cluster_id += 1

  return output_cluster_ids

In [ ]:
clusterIDs = get_cluster_by_number(result1, 10)
print(clusterIDs)

In [ ]:
plt.hist(clusterIDs)
plt.grid()

#### シルエット係数
a を「同じクラスタに属するメンバ間の距離の平均」、b を「異なるクラスタに属するメンバ間の距離の平均」としたときに、

$
\dfrac{b - a}{max(b, a)}
$

で表されます。

シルエット係数が高いほど、よく分割できている（クラスタ間距離に比べ、クラスタ内距離が十分に短い）ことを示します

In [ ]:
import math
def silhouette_coefficient(clusters, df):
  a_same = []
  b_diff = []
  for i, j in enumerate(clusters):
    vec1 = df.iloc[i, :].values
    for k, l in enumerate(clusters):
      if i < k:
        vec2 = df.iloc[k, :].values
        dist = 0.
        for v1, v2 in zip(vec1, vec2):
          dist += (v1 - v2) ** 2
        dist = math.sqrt(dist)    # 距離の定義を「ユークリッド距離」とした場合
        if j == l: # same cluster
          a_same.append(dist)
        else: # different cluster
          b_diff.append(dist)
  a = sum(a_same) / len(a_same)
  b = sum(b_diff) / len(b_diff)
  return (b - a) / max(b, a)

In [ ]:
silhouette_coefficient(clusterIDs, df)

#### シルエット係数のクラスタ数依存性

In [ ]:
x = []
y = []
for i in range(2, len(df)):
  x.append(i)
  y.append(silhouette_coefficient(get_cluster_by_number(result1, i), df))

plt.plot(x, y)
plt.xlabel("Num of clusters")
plt.ylabel("silhouette coefficient")
plt.grid()
plt.show()

#### 計算の高速化
- 計算した距離は distance_matrix に格納して、それを必要に応じて読み込む形に改善

In [ ]:
import math
def get_distance_matrix(df):
  distance_matrix = []
  for i in range(len(df)):
    vec1 = df.iloc[i, :].values
    distance_array = []
    for j in range(len(df)):
      vec2 = df.iloc[j, :].values
      dist = 0.
      for v1, v2 in zip(vec1, vec2):
        dist += (v1 - v2) ** 2
      distance_array.append(math.sqrt(dist))
    distance_matrix.append(distance_array)
  return distance_matrix

In [ ]:
def silhouette_coefficient2(clusters, distance_matrix):
  a_same = []
  b_diff = []
  for i, j in enumerate(clusters):
    for k, l in enumerate(clusters):
      if i < k:
        dist = distance_matrix[i][k]
        if j == l:  # same cluster
          a_same.append(dist)
        else:   # different cluster
          b_diff.append(dist)
  a = sum(a_same) / len(a_same)
  b = sum(b_diff) / len(b_diff)
  return (b - a) / max(b, a)

In [ ]:
distance_matrix = get_distance_matrix(df)

x = []
y = []
for i in range(2, len(df)):
  x.append(i)
  y.append(silhouette_coefficient2(get_cluster_by_number(result1, i), distance_matrix))

plt.plot(x, y)
plt.xlabel("Num of clusters")
plt.ylabel("silhouette coefficient")
plt.grid()
plt.show()

#### クラスタに分割して中身を見る

In [ ]:
clusterIDs = get_cluster_by_number(result1, 10)

In [ ]:
for i in range(max(clusterIDs) + 1):
  cluster = []
  for j, k in enumerate(clusterIDs):
    if i == k:
      cluster.append(j)
  plt.title("Cluster {}: {} samples".format(i + 1, len(cluster)))

  for j in cluster:
    plt.plot(dfs.iloc[j, :], label=dfs.index[j])

  plt.legend(loc = 'center right',
             bbox_to_anchor = (0.8, 0.5, 0.5, 0.1),
             borderaxespad = 0.0)
  plt.grid()
  plt.show()